A *degree n pillowcase-tiled surface* is represented by a tuple of four elements $(a,b,c,d) \in \text{Sym}(n)^4$
such that $abcd = 1$. A PTS is *1-cylinder* if every element of its braid group orbit has the property that $ab$ is an $n$-cycle. Our goal is to find non-cyclic 1-cylinder PTS, in other words, tuples such that 
* for every $g \in B_4$, if $g\cdot(a,b,c,d) = (a',b',c',d')$ then $a'b'$ is an $n$-cycle, and 
* $\langle a,b,c,d\rangle$ is a non-cyclic subgroup of $\text{Sym}(n)$.  

We're particularly interested if there are examples where the permutations $a,b,c,d$ do not include the identity permutation.

We use the class `Constellation` from the `sage::combinat` library. A *constellation* is a tuple of permutations in $\text{Sym(n)}$ whose product is the identity. The *length* of a constellation is the number of elements in the tuple,
and the *degree* is the $n$ of the symmetric group the permutations come from.
The `constellation` library by default only returns connected constellations, i.e. 
constellations whose permutations have a connected orbit when acting on 
$\{1,\dots,n\}$. In order to be a 1-cylinder surface, a PTS must be represented by a connected 
constellation, so this is ok.

In [ ]:
from sage.all import SymmetricGroup
from sage.combinat import constellation

n = 5
Sn = SymmetricGroup(n)
all_pts = Constellations(4,n) # only returns connected constellations
horizontal_one_cyl_set = set()
for pts in all_pts:
    if (pts.g(0)*pts.g(1)).cycle_type() == [n]:
        horizontal_one_cyl_set.add(pts)
print(f'Total number of degree {n} PTS: {len(all_pts)}')
print(f'Total number of degree {n} PTS with a horizontal cyl: {len(horizontal_one_cyl_set)}')

Total number of degree 5 PTS: 1647384
Total number of degree 5 PTS with a horizontal cyl: 345600


In [72]:
class PTS:
    # n is an integer > 1, constellation is an instance of the Constellation 
    # class on Sym(n)
    def __init__(self, n, constellation):
        self.n = n 
        self.constellation = constellation
    
    def g(self,i):
        return self.constellation.g(i)
    
    def braid_group_orbit(self):
        orbit = []
        for t in self.constellation.braid_group_orbit().vertices():
            orbit.append(PTS(self.n, t))
        
        return orbit

    def is_cyclic(self):
        G = SymmetricGroup(self.n).subgroup([self.g(0), self.g(1), self.g(2), self.g(3)])
        return G.is_cyclic()
    
    # check if there is one cylinder in the horizontal direction
    def check_horizontal_direction(self):
        if (self.g(0)*self.g(1)).cycle_type() == [self.n]:
            return True 
        else:
            return False
    
    def is_one_cyl_surface(self):
        orbit = self.braid_group_orbit()
        for t in orbit: # t not nec = pts, but will be isom. constellations
            if not t.check_horizontal_direction():
                return False
        return True
    
    def contains_identity(self):
        for i in range(0,4):
            if self.g(i).cycle_type() == ([1]*self.n):
                return True 
        return False

    def __str__(self):
        return f'({self.g(0)},  {self.g(1)},  {self.g(2)},  {self.g(3)})'

    def verbose_print(self):
        print('PTS defined by:')
        print(f' a: {self.g(0)}')
        print(f' b: {self.g(1)}')
        print(f' c: {self.g(2)}')
        print(f' d: {self.g(3)}\n')
    
        print(f'cycle type: {self.constellation.profile()}')
        print(f'is cyclic: {self.is_cyclic()}')
        print(f'ab = {self.g(0)*self.g(1)}')
        print(f'is 1-cylinder: {self.is_one_cyl_surface()}')
        print(f'Braid group orbit:')
        for t in self.braid_group_orbit():
            print(t)
            print(f'a\'b\'={t.g(0)*t.g(1)}')
        print(f'\n')


In [77]:
one_cylinder = set() # all 1-cylinder surfaces

for pts in horizontal_one_cyl_set:
    if PTS(n,pts).is_one_cyl_surface():
        one_cylinder.add(pts)

print(f'Total number of degree {n} one-cylinder PTS: {len(one_cylinder)}')

Total number of degree 5 one-cylinder PTS: 69120


In [ ]:
cyclic_one_cyl = set()
non_cyclic_one_cyl = set()
for pts in one_cylinder:
    pts = PTS(n,pts)
    cyclic_one_cyl.add(pts) if pts.is_cyclic() else non_cyclic_one_cyl.add(pts)

print(f'Number of degree {n} one-cylinder PTS that are: \n cyclic: {len(cyclic_one_cyl)}')

non_cyclic_one_cylinder_non_identity = set()
identity = set()
for pts in non_cyclic_one_cyl:
    identity.add(pts) if pts.contains_identity() else non_cyclic_one_cylinder_non_identity.add(pts)
        
print(f' non-cyclic with identity perm: {len(identity)}')
print(f' non-cyclic without identity perm: {len(non_cyclic_one_cylinder_non_identity)}\n')

cycle_type_dict = {}
for pts in non_cyclic_one_cyl:
    key = tuple(sorted(pts.constellation.profile()))
    if not cycle_type_dict.get(key)==None:
        cycle_type_dict.get(key).append(pts)
    else:
        cycle_type_dict.update({key: [pts]})
print(f'Number of cycle types up to permutation: {len(cycle_type_dict.keys())}')


Number of degree 5 one-cylinder PTS that are: 
 cyclic: 480
 non-cyclic with identity perm: 1920
 non-cyclic without identity perm: 66720

Number of cycle types up to permutation: 29


In [74]:
for (key,value) in cycle_type_dict.items():
    print(f'{len(value)} pts of type {key}')
    example_pts = value[0]
    print('example:')
    example_pts.verbose_print()

6000 pts of type ([3, 1, 1], [3, 1, 1], [5], [5])
example:
PTS defined by:
 a: (1,3,4)
 b: (1,3,4,2,5)
 c: (1,2,5,3,4)
 d: (2,5,3)

cycle type: ([3, 1, 1], [5], [5], [3, 1, 1])
is cyclic: False
ab = (1,4,3,2,5)
is 1-cylinder: True
Braid group orbit:
((3,4,5),  (1,2,3,4,5),  (1,2,4,5,3),  (1,2,4))
a'b'=(1,2,3,5,4)
((1,2,3,4,5),  (3,4,5),  (1,3,4,2,5),  (1,3,5))
a'b'=(1,2,4,3,5)
((3,4,5),  (1,2,3,4,5),  (1,4,5,2,3),  (1,2,3))
a'b'=(1,2,3,5,4)
((3,4,5),  (1,2,3,4,5),  (1,2,4),  (1,5,3,2,4))
a'b'=(1,2,3,5,4)


2880 pts of type ([3, 1, 1], [5], [5], [5])
example:
PTS defined by:
 a: (1,5,2,4,3)
 b: (1,4,3,5,2)
 c: (1,5,3,2,4)
 d: (1,3,4)

cycle type: ([5], [5], [5], [3, 1, 1])
is cyclic: False
ab = (1,2,3,4,5)
is 1-cylinder: True
Braid group orbit:
((1,2,3,4,5),  (1,2,4,5,3),  (1,3,4,2,5),  (1,3,2))
a'b'=(1,4,3,5,2)
((1,2,3,4,5),  (1,2,4,5,3),  (1,3,2,5,4),  (2,4,3))
a'b'=(1,4,3,5,2)
((1,2,3,4,5),  (1,2,4,5,3),  (1,3,2),  (1,5,3,2,4))
a'b'=(1,4,3,5,2)
((2,3,4),  (1,2,3,5,4),  (1,4,3,2,5),  